# Infer-10-Crowdsourcing : Agregation de Labels et Fiabilite

**Serie** : Programmation Probabiliste avec Infer.NET (10/12)  
**Duree estimee** : 55 minutes  
**Prerequis** : Infer-9-Topic-Models

---

## Objectifs

- Comprendre le probleme de l'agregation de labels
- Implementer le modele Honest Worker
- Construire le modele Biased Worker avec matrice de confusion
- Explorer le modele hierarchique Community

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-9-Topic-Models](Infer-9-Topic-Models.ipynb) | [Infer-11-Sequences](Infer-11-Sequences.ipynb) |

---

## 1. Configuration

Nous chargeons les packages pour les modeles de crowdsourcing. Ces modeles probabilistes permettent d'agreger des annotations provenant de multiples travailleurs de fiabilites variables, tout en estimant simultanement la verite latente et la qualite de chaque annotateur.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Le Probleme du Crowdsourcing

### Contexte

On veut annoter un grand nombre d'images (spam/non-spam, sentiment, etc.) en utilisant des travailleurs non-experts sur des plateformes comme Amazon Mechanical Turk.

### Defis

| Defi | Description |
|------|-------------|
| **Fiabilite variable** | Certains travailleurs sont meilleurs que d'autres |
| **Biais** | Certains ont tendance a repondre toujours la meme chose |
| **Verite inconnue** | On ne connait pas le vrai label |
| **Cout** | Limiter le nombre d'annotations par item |

### Solution

Modele probabiliste qui estime simultanement :
- Le vrai label de chaque item
- La qualite de chaque travailleur

## 3. Modele Honest Worker

### Hypothese

Chaque travailleur a une **capacite** (probabilite de donner la bonne reponse).

### Modele

$$\text{capacite}_w \sim \text{Beta}(\alpha, \beta)$$

$$P(\text{label}_w = \text{vrai label}) = \text{capacite}_w$$
$$P(\text{label}_w \neq \text{vrai label}) = \frac{1 - \text{capacite}_w}{K-1}$$

ou K est le nombre de classes.

In [2]:
// Donnees de crowdsourcing
// 5 items, 4 workers, 2 classes (0 ou 1)

int nItems = 5;
int nWorkers = 4;
int nClasses = 2;

// Labels donnes par chaque worker pour chaque item (-1 = pas d'annotation)
int[,] labels = {
    // W1  W2  W3  W4
    {  1,  1,  1,  0 },  // Item 0 : consensus 1
    {  0,  0,  0,  0 },  // Item 1 : consensus 0
    {  1,  1,  0,  1 },  // Item 2 : majorite 1
    {  0,  1,  0,  0 },  // Item 3 : majorite 0
    {  1,  0,  1,  1 }   // Item 4 : majorite 1
};

// Vrais labels (pour evaluation - inconnus du modele)
int[] vraisLabels = { 1, 0, 1, 0, 1 };

Console.WriteLine("=== Donnees Crowdsourcing ===");
Console.WriteLine("\nLabels donnes par les workers :");
Console.Write("        ");
for (int w = 0; w < nWorkers; w++) Console.Write($"W{w+1} ");
Console.WriteLine("| Vrai");

for (int i = 0; i < nItems; i++)
{
    Console.Write($"Item {i} : ");
    for (int w = 0; w < nWorkers; w++)
    {
        Console.Write($" {labels[i, w]} ");
    }
    Console.WriteLine($" |  {vraisLabels[i]}");
}

=== Donnees Crowdsourcing ===

Labels donnes par les workers :
        W1 W2 W3 W4 | Vrai
Item 0 :  1  1  1  0  |  1
Item 1 :  0  0  0  0  |  0
Item 2 :  1  1  0  1  |  1
Item 3 :  0  1  0  0  |  0
Item 4 :  1  0  1  1  |  1


In [3]:
// Modele Honest Worker simplifie (pour un item)

int itemIdx = 2;  // Item avec desaccord
int[] labelsItem = { labels[itemIdx, 0], labels[itemIdx, 1], labels[itemIdx, 2], labels[itemIdx, 3] };

// Prior sur le vrai label (uniforme)
Variable<int> vraiLabel = Variable.DiscreteUniform(nClasses).Named("vraiLabel");

// Capacites des workers
Range workerRange = new Range(nWorkers).Named("worker");
VariableArray<double> capacite = Variable.Array<double>(workerRange).Named("capacite");
capacite[workerRange] = Variable.Beta(2, 1).ForEach(workerRange);  // Prior : plus de 50%

// Labels observes
VariableArray<int> labelObs = Variable.Array<int>(workerRange).Named("labelObs");

using (Variable.ForEach(workerRange))
{
    Variable<bool> estCorrect = Variable.Bernoulli(capacite[workerRange]);
    using (Variable.If(estCorrect))
    {
        labelObs[workerRange] = vraiLabel;  // Repond correctement
    }
    using (Variable.IfNot(estCorrect))
    {
        labelObs[workerRange] = Variable.DiscreteUniform(nClasses);  // Reponse aleatoire
    }
}

labelObs.ObservedValue = labelsItem;

InferenceEngine moteurHW = new InferenceEngine();
moteurHW.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Discrete vraiLabelPost = moteurHW.Infer<Discrete>(vraiLabel);
Beta[] capacitePost = moteurHW.Infer<Beta[]>(capacite);

Console.WriteLine($"\n=== Honest Worker pour Item {itemIdx} ===");
Console.WriteLine($"Labels observes : {string.Join(", ", labelsItem)}");
Console.WriteLine($"Vrai label : {vraisLabels[itemIdx]}\n");

Console.WriteLine($"P(vrai label = 0) = {vraiLabelPost.GetProbs()[0]:F3}");
Console.WriteLine($"P(vrai label = 1) = {vraiLabelPost.GetProbs()[1]:F3}");

Console.WriteLine("\nCapacites des workers :");
for (int w = 0; w < nWorkers; w++)
{
    Console.WriteLine($"  Worker {w+1} : {capacitePost[w].GetMean():F3}");
}

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50

=== Honest Worker pour Item 2 ===
Labels observes : 1, 1, 0, 1
Vrai label : 1

P(vrai label = 0) = 0,038
P(vrai label = 1) = 0,962

Capacites des workers :
  Worker 1 : 0,692
  Worker 2 : 0,692
  Worker 3 : 0,508
  Worker 4 : 0,692


### Analyse du modèle Honest Worker

**Item analysé** : Item 2 avec labels [1, 1, 0, 1] (vrai label = 1)

| Mesure | Valeur | Interprétation |
|--------|--------|----------------|
| **P(vrai=1)** | 0.962 | Forte confiance malgré le désaccord |
| **P(vrai=0)** | 0.038 | Très faible probabilité |
| **Capacité W1, W2, W4** | 0.692 | Workers en accord avec la majorité |
| **Capacité W3** | 0.508 | Worker minoritaire ≈ aléatoire |

**Observations clés** :

1. **Consensus bayésien** : 3/4 workers disent "1" → P(vrai=1) = 96.2%, bien plus que le 75% du vote brut

2. **Estimation de capacité** :
   - Workers majoritaires : capacité ~0.69 (supérieure au prior Beta(2,1) = 0.67)
   - Worker minoritaire : capacité ~0.51 (proche du hasard)

3. **Effet du prior** : Beta(2,1) encode l'hypothèse que les workers sont "honnêtes" (> 50% correct)

**Pourquoi P(vrai=1) > 75% ?**

Le modèle bayésien pondère les votes par la capacité estimée. Les workers en accord renforcent mutuellement leur crédibilité, tandis que W3 minoritaire perd en influence.

## 4. Modele Biased Worker

### Amelioration

Au lieu d'une simple capacite, chaque worker a une **matrice de confusion** qui capture ses biais.

### Modele

$$\text{confusion}_{w}[c] \sim \text{Dirichlet}(\alpha)$$

$$P(\text{label}_w = l | \text{vrai} = c) = \text{confusion}_w[c][l]$$

In [4]:
// Modele Biased Worker simplifie

// Pour un worker specifique, estimer sa matrice de confusion
// etant donne plusieurs annotations avec vrais labels connus

// Worker 3 (W4) semble avoir fait des erreurs
int[] w4Labels = { labels[0, 3], labels[1, 3], labels[2, 3], labels[3, 3], labels[4, 3] };
// W4 labels : 0, 0, 1, 0, 1
// Vrais     : 1, 0, 1, 0, 1
// W4 se trompe sur item 0 (vrai=1, dit 0)

// Prior sur la matrice de confusion (Dirichlet par ligne)
Range classRange = new Range(nClasses).Named("class");

VariableArray<Vector> confusion = Variable.Array<Vector>(classRange).Named("confusion");
double[] dirichletPrior = { 10, 1 };  // Prior : plus de chances de repondre correctement

using (Variable.ForEach(classRange))
{
    // Pour chaque vrai label, distribution sur les reponses possibles
    confusion[classRange] = Variable.Dirichlet(dirichletPrior);
}

// Observations
Range itemRange2 = new Range(nItems).Named("item");
VariableArray<int> vraiLabelsVar = Variable.Array<int>(itemRange2).Named("vrai");
VariableArray<int> workerLabelsVar = Variable.Array<int>(itemRange2).Named("workerLabel");

// IMPORTANT: SetValueRange est necessaire pour utiliser Variable.Switch()
vraiLabelsVar.SetValueRange(classRange);

using (Variable.ForEach(itemRange2))
{
    using (Variable.Switch(vraiLabelsVar[itemRange2]))
    {
        workerLabelsVar[itemRange2] = Variable.Discrete(confusion[vraiLabelsVar[itemRange2]]);
    }
}

vraiLabelsVar.ObservedValue = vraisLabels;
workerLabelsVar.ObservedValue = w4Labels;

InferenceEngine moteurBW = new InferenceEngine(new ExpectationPropagation());
moteurBW.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Dirichlet[] confusionPost = moteurBW.Infer<Dirichlet[]>(confusion);

Console.WriteLine("=== Biased Worker (W4) ===");
Console.WriteLine("\nMatrice de confusion estimee :");
Console.WriteLine("           Repond 0  Repond 1");
for (int c = 0; c < nClasses; c++)
{
    Vector probs = confusionPost[c].GetMean();
    Console.WriteLine($"Vrai = {c} :   {probs[0]:F2}       {probs[1]:F2}");
}

Compiling model...done.
=== Biased Worker (W4) ===

Matrice de confusion estimee :
           Repond 0  Repond 1
Vrai = 0 :   0,92       0,08
Vrai = 1 :   0,79       0,21


### Analyse de la matrice de confusion (W4)

**Données** : W4 labels = [0, 0, 1, 0, 1], Vrais = [1, 0, 1, 0, 1]
- Erreur sur Item 0 : vrai=1, W4 dit 0
- Correct sur Items 1, 2, 3, 4

**Matrice de confusion estimée** :

|  | Répond 0 | Répond 1 |
|--|----------|----------|
| **Vrai=0** | 0.92 | 0.08 |
| **Vrai=1** | 0.79 | 0.21 |

**Interprétation** :

1. **Biais détecté** : W4 a tendance à répondre "0" même quand le vrai label est "1"
   - Vrai=0 → 92% correct (bon)
   - Vrai=1 → seulement 21% correct (mauvais)

2. **Asymétrie** : Ce n'est pas juste une capacité faible, c'est un **biais systématique** vers la classe 0

3. **Prior Dirichlet(10,1)** : Encode une préférence pour les réponses correctes, mais les données dominent

**Avantage du modèle Biased Worker** :

Contrairement à Honest Worker qui suppose une erreur symétrique, ce modèle capture que W4 :
- Reconnaît bien la classe 0
- Mais confond souvent la classe 1 avec la classe 0

## 5. Agregation de Tous les Items

In [5]:
// Agregation par vote majoritaire (baseline)

Console.WriteLine("=== Comparaison des Methodes ===");
Console.WriteLine("\nVote majoritaire vs Vrai label :");

int correctMajority = 0;

for (int i = 0; i < nItems; i++)
{
    int count0 = 0, count1 = 0;
    for (int w = 0; w < nWorkers; w++)
    {
        if (labels[i, w] == 0) count0++;
        else count1++;
    }
    
    int majority = count0 > count1 ? 0 : 1;
    bool correct = majority == vraisLabels[i];
    if (correct) correctMajority++;
    
    Console.WriteLine($"Item {i} : Majoritaire={majority}, Vrai={vraisLabels[i]} {(correct ? "OK" : "ERREUR")}");
}

Console.WriteLine($"\nPrecision vote majoritaire : {correctMajority}/{nItems} = {100.0 * correctMajority / nItems:F0}%");

=== Comparaison des Methodes ===

Vote majoritaire vs Vrai label :
Item 0 : Majoritaire=1, Vrai=1 OK
Item 1 : Majoritaire=0, Vrai=0 OK
Item 2 : Majoritaire=1, Vrai=1 OK
Item 3 : Majoritaire=0, Vrai=0 OK
Item 4 : Majoritaire=1, Vrai=1 OK

Precision vote majoritaire : 5/5 = 100%


### Analyse du vote majoritaire

**Résultat** : 100% de précision (5/5 items corrects)

| Item | Votes (0:1) | Majorité | Vrai | Résultat |
|------|-------------|----------|------|----------|
| 0 | 1:3 | 1 | 1 | OK |
| 1 | 4:0 | 0 | 0 | OK |
| 2 | 1:3 | 1 | 1 | OK |
| 3 | 3:1 | 0 | 0 | OK |
| 4 | 1:3 | 1 | 1 | OK |

**Observations** :

1. **Performance parfaite** : Dans ce petit exemple, le vote majoritaire suffit
2. **Cas favorables** : Tous les items ont un consensus clair (3:1 ou 4:0)
3. **Limitation** : Avec des cas 2:2, le vote majoritaire est indécis

**Quand le modèle probabiliste est meilleur** :

1. **Peu d'annotations** : 1-2 votes par item → besoin de pondérer par qualité
2. **Workers biaisés** : Vote majoritaire ignore les biais systématiques
3. **Items difficiles** : Les modèles estiment aussi l'incertitude
4. **Données manquantes** : Annotations incomplètes gérées naturellement

**Note** : La vraie valeur des modèles de crowdsourcing apparaît sur de grands datasets avec workers hétérogènes.

## 6. Modele Community (Hierarchique)

### Idee

Les workers appartiennent a des **communautes** avec des caracteristiques similaires.

### Structure

```
Communaute[c] : matrice de confusion partagee
    |
    v
Worker[w] : appartient a communaute z[w]
    |
    v
Label[w,i] : genere selon confusion[z[w]]
```

In [6]:
// Modele Community simplifie

// Simulons 2 communautes :
// - Experts (haute precision)
// - Spammeurs (repondent aleatoirement)

int nCommunities = 2;

// Distribution sur les communautes
Variable<Vector> pCommunity = Variable.Dirichlet(new double[] { 1, 1 }).Named("pCommunity");

Range communityRange = new Range(nCommunities).Named("community");
Range workerRange3 = new Range(nWorkers).Named("worker");

// Capacite par communaute
VariableArray<double> communityCapacity = Variable.Array<double>(communityRange).Named("communityCapacity");
communityCapacity[communityRange] = Variable.Beta(2, 1).ForEach(communityRange);

// Assignation des workers aux communautes
VariableArray<int> workerCommunity = Variable.Array<int>(workerRange3).Named("workerCommunity");
workerCommunity[workerRange3] = Variable.Discrete(pCommunity).ForEach(workerRange3);

Console.WriteLine("=== Modele Community ===");
Console.WriteLine("\nHypothese : 2 communautes (Experts / Spammeurs)");
Console.WriteLine("\nLe modele infere l'appartenance de chaque worker.");

=== Modele Community ===

Hypothese : 2 communautes (Experts / Spammeurs)

Le modele infere l'appartenance de chaque worker.


## 7. Apprentissage Actif

### Objectif

Choisir **quels items** faire annoter et par **quels workers** pour maximiser l'information gagnee.

### Strategies

| Strategie | Description |
|-----------|-------------|
| **Uncertainty sampling** | Annoter les items les plus incertains |
| **Worker evaluation** | Tester les workers sur des items connus |
| **Information gain** | Maximiser la reduction d'entropie |

In [7]:
// Selection d'items basee sur l'incertitude

Console.WriteLine("=== Apprentissage Actif ===");
Console.WriteLine("\nSelection des items les plus incertains :");

// Calculer l'incertitude (entropie) pour chaque item base sur le vote majoritaire
var incertitudes = new List<(int item, double entropie)>();

for (int i = 0; i < nItems; i++)
{
    int count0 = 0, count1 = 0;
    for (int w = 0; w < nWorkers; w++)
    {
        if (labels[i, w] == 0) count0++;
        else count1++;
    }
    
    double p0 = (double)count0 / nWorkers;
    double p1 = (double)count1 / nWorkers;
    
    // Entropie binaire
    double entropie = 0;
    if (p0 > 0) entropie -= p0 * Math.Log2(p0);
    if (p1 > 0) entropie -= p1 * Math.Log2(p1);
    
    incertitudes.Add((i, entropie));
    Console.WriteLine($"Item {i} : votes 0:{count0} / 1:{count1}, entropie = {entropie:F3}");
}

var prioritaire = incertitudes.OrderByDescending(x => x.entropie).First();
Console.WriteLine($"\n=> Item {prioritaire.item} devrait etre annote en priorite (entropie max)");

=== Apprentissage Actif ===

Selection des items les plus incertains :
Item 0 : votes 0:1 / 1:3, entropie = 0,811
Item 1 : votes 0:4 / 1:0, entropie = 0,000
Item 2 : votes 0:1 / 1:3, entropie = 0,811
Item 3 : votes 0:3 / 1:1, entropie = 0,811
Item 4 : votes 0:1 / 1:3, entropie = 0,811

=> Item 0 devrait etre annote en priorite (entropie max)


### Analyse de l'apprentissage actif

**Résultats d'entropie** :

| Item | Votes (0:1) | Entropie | Incertitude |
|------|-------------|----------|-------------|
| 1 | 4:0 | 0.000 | **Nulle** (consensus total) |
| 0, 2, 3, 4 | 1:3 ou 3:1 | 0.811 | **Égale** (même distribution 75/25) |

**Observations** :

1. **Item 1 exclu** : Consensus parfait → pas besoin d'annotation supplémentaire
2. **Items 0, 2, 3, 4 équivalents** : Même niveau d'incertitude (1 dissident sur 4)
3. **Choix arbitraire** : L'algorithme sélectionne Item 0 (premier dans l'ordre)

**Stratégies d'amélioration** :

| Critère | Description |
|---------|-------------|
| **Entropie** | Annoter les items les plus incertains |
| **Utilité attendue** | Combiner incertitude × importance de l'item |
| **Diversité workers** | Solliciter des workers différents |
| **Gold questions** | Mélanger des items connus pour évaluer les workers |

**Note** : Avec une entropie max de 1.0 (50/50), une entropie de 0.811 représente une incertitude significative (équivalent à ~20% de désaccord).

## 8. Exercice : Crowdsourcing d'Images

### Enonce

Simulez un scenario de classification d'images (chat/chien) avec 8 workers de qualites variables.

In [8]:
// EXERCICE : Crowdsourcing d'images

int nImages = 10;
int nAnnotateurs = 8;
Random rng = new Random(42);

// Vrais labels (0=chat, 1=chien)
int[] vraisLabelsImg = Enumerable.Range(0, nImages).Select(i => rng.Next(2)).ToArray();

// Qualites des annotateurs (differentes)
double[] qualites = { 0.95, 0.90, 0.85, 0.80, 0.70, 0.60, 0.55, 0.50 };

// Generer les annotations
int[,] annotations = new int[nImages, nAnnotateurs];

for (int i = 0; i < nImages; i++)
{
    for (int a = 0; a < nAnnotateurs; a++)
    {
        if (rng.NextDouble() < qualites[a])
            annotations[i, a] = vraisLabelsImg[i];  // Correct
        else
            annotations[i, a] = 1 - vraisLabelsImg[i];  // Erreur
    }
}

Console.WriteLine("=== Crowdsourcing Images (Chat=0 / Chien=1) ===");
Console.WriteLine($"\nQualites vraies des annotateurs : {string.Join(", ", qualites.Select(q => $"{q:F2}"))}\n");

// Evaluation par vote majoritaire
int correctes = 0;
for (int i = 0; i < nImages; i++)
{
    int votes1 = Enumerable.Range(0, nAnnotateurs).Count(a => annotations[i, a] == 1);
    int prediction = votes1 > nAnnotateurs / 2 ? 1 : 0;
    if (prediction == vraisLabelsImg[i]) correctes++;
}

Console.WriteLine($"Precision vote majoritaire : {100.0 * correctes / nImages:F0}%");

// Estimation des qualites basee sur le consensus
Console.WriteLine("\nEstimation des qualites (accord avec majorite) :");
for (int a = 0; a < nAnnotateurs; a++)
{
    int accords = 0;
    for (int i = 0; i < nImages; i++)
    {
        int votes1 = Enumerable.Range(0, nAnnotateurs).Count(x => annotations[i, x] == 1);
        int majorite = votes1 > nAnnotateurs / 2 ? 1 : 0;
        if (annotations[i, a] == majorite) accords++;
    }
    double qualiteEstimee = (double)accords / nImages;
    Console.WriteLine($"  Annotateur {a+1} : vraie={qualites[a]:F2}, estimee={qualiteEstimee:F2}");
}

=== Crowdsourcing Images (Chat=0 / Chien=1) ===

Qualites vraies des annotateurs : 0,95, 0,90, 0,85, 0,80, 0,70, 0,60, 0,55, 0,50

Precision vote majoritaire : 100%

Estimation des qualites (accord avec majorite) :
  Annotateur 1 : vraie=0,95, estimee=1,00
  Annotateur 2 : vraie=0,90, estimee=1,00
  Annotateur 3 : vraie=0,85, estimee=1,00
  Annotateur 4 : vraie=0,80, estimee=0,90
  Annotateur 5 : vraie=0,70, estimee=1,00
  Annotateur 6 : vraie=0,60, estimee=0,80
  Annotateur 7 : vraie=0,55, estimee=0,70
  Annotateur 8 : vraie=0,50, estimee=0,50


### Analyse de l'exercice crowdsourcing

**Configuration** : 10 images, 8 annotateurs avec qualités de 0.50 à 0.95

**Résultats** :

| Annotateur | Qualité vraie | Qualité estimée | Écart |
|------------|---------------|-----------------|-------|
| 1 | 0.95 | 1.00 | +0.05 |
| 2 | 0.90 | 1.00 | +0.10 |
| 3 | 0.85 | 1.00 | +0.15 |
| 4 | 0.80 | 0.90 | +0.10 |
| 5 | 0.70 | 1.00 | +0.30 |
| 6 | 0.60 | 0.80 | +0.20 |
| 7 | 0.55 | 0.70 | +0.15 |
| 8 | 0.50 | 0.50 | 0.00 |

**Observations** :

1. **Surestimation systématique** : L'estimation par accord avec la majorité surestime la qualité
2. **Raison** : Le vote majoritaire est généralement correct → s'y accorder donne un bon score
3. **Annotateur 8 (0.50)** : Seul cas bien estimé car vraiment aléatoire
4. **Annotateur 5 (0.70 → 1.00)** : Surestimation maximale car chance d'être du bon côté

**Limites de l'estimation par consensus** :

- Ne détecte pas les **biais corrélés** entre workers
- Nécessite des **gold questions** (items de vérité connue) pour calibrer
- Un modèle Honest Worker donnerait des estimations plus conservatrices

## 9. Resume

| Concept | Description |
|---------|-------------|
| **Honest Worker** | Capacite unique par worker |
| **Biased Worker** | Matrice de confusion par worker |
| **Community** | Workers groupes en communautes |
| **Apprentissage actif** | Selection optimale des annotations |
| **Gold standard** | Items avec vrai label connu pour evaluation |

---

## Prochaine etape

Dans [Infer-11-Sequences](Infer-11-Sequences.ipynb), nous explorerons :

- Les Hidden Markov Models (HMM)
- L'algorithme de Viterbi
- L'application au motif finding en bioinformatique